This analysis estimates how often an asset breaches a specified downside threshold over a configurable holding window (e.g., 1-week, 2-week, 3-week, monthly).

For each window, returns are computed end-to-end as:

Return = Price_end / Price_start − 1


The script reports the frequency with which returns fall below a chosen threshold (e.g., −2.5%, −5%, −9%). This provides a simple, strike-agnostic proxy for how often a breakeven level would historically be violated over different time horizons.

Results are purely price-based and intended as a first-pass downside risk sanity check for short-dated option strategies.

In [36]:
import pandas as pd
import yfinance as yf
import numpy as np

# -----------------------------
# Parameters (edit these)
# -----------------------------
ticker = "AMZ"

# Timeline
USE_YEARS = False
years = 10

start_date = "2010-01-01"
end_date   = None  # None = today

# Threshold + window
threshold = -0.078          # e.g., -2.5%
WINDOW = "1W"               # "1W", "2W", "4W", "1M", "3M"
ANCHOR = "W-FRI"            # "W-FRI" (week ends Fri) or "W" etc.
RETURN_TYPE = "simple"      # "simple" or "log"

# -----------------------------
# Load daily data
# -----------------------------
if USE_YEARS:
    hist = yf.Ticker(ticker).history(period=f"{years}y", auto_adjust=True)
else:
    hist = yf.Ticker(ticker).history(start=start_date, end=end_date, auto_adjust=True)

required_cols = {"Close"}
if not required_cols.issubset(hist.columns):
    raise ValueError(f"Missing columns. Found: {list(hist.columns)}")

hist = hist[hist.index.dayofweek < 5].copy()  # defensive weekdays only

# -----------------------------
# Build windowed returns (end-to-end over the window)
# -----------------------------
# Resample to window endpoints (e.g., weekly Friday closes, 2W Friday closes, monthly closes)
close = hist["Close"].dropna()

# NOTE: "1M" in pandas is calendar month end. "4W" is 4-week blocks.
window_close = close.resample(WINDOW).last()

if RETURN_TYPE == "log":
    window_ret = np.log(window_close / window_close.shift(1))
else:
    window_ret = window_close / window_close.shift(1) - 1

window_ret = window_ret.dropna().rename("ret")

# -----------------------------
# Statistics
# -----------------------------
total_periods = len(window_ret)
count_breach = int((window_ret <= threshold).sum())
prob_breach = count_breach / total_periods if total_periods > 0 else np.nan

print(f"Ticker: {ticker}")
if USE_YEARS:
    print(f"Window: last {years} years")
else:
    print(f"Window: {start_date} to {end_date or 'today'}")

print(f"\nReturn horizon: {WINDOW}")
print(f"Total periods analyzed: {total_periods}")
print(f"Periods with return ≤ {threshold:.1%}: {count_breach} ({prob_breach:.2%})")

print("\nSummary of windowed returns:")
print(window_ret.describe(percentiles=[0.01, 0.05, 0.10]))

print(f"\nMost recent periods breaching {threshold:.1%}:")
print(
    window_ret[window_ret <= threshold]
    .tail(10)
    .to_frame()
    .assign(ret_pct=lambda x: (x["ret"] * 100).round(2))
)


Ticker: AMZ
Window: 2010-01-01 to today

Return horizon: 1W
Total periods analyzed: 411
Periods with return ≤ -7.8%: 8 (1.95%)

Summary of windowed returns:
count    411.000000
mean       0.006399
std        0.040277
min       -0.153710
1%        -0.097909
5%        -0.057829
10%       -0.042989
50%        0.004797
max        0.169993
Name: ret, dtype: float64

Most recent periods breaching -7.8%:
                                ret  ret_pct
Date                                        
2010-07-04 00:00:00-04:00 -0.113653   -11.37
2011-08-07 00:00:00-04:00 -0.093820    -9.38
2011-08-21 00:00:00-04:00 -0.086773    -8.68
2011-10-30 00:00:00-04:00 -0.109932   -10.99
2014-07-27 00:00:00-04:00 -0.098364    -9.84
2016-01-10 00:00:00-05:00 -0.107526   -10.75
2016-02-07 00:00:00-05:00 -0.153710   -15.37
2017-08-06 00:00:00-04:00 -0.090845    -9.08


In [37]:
# ============================================================
# Efficient "Put Credit Spread / Tail Breach" add-on (ARCH/GARCH)
# Drop this in your LAST cell. It reuses your existing variables:
#   ticker, USE_YEARS, years, start_date, end_date
#   threshold, WINDOW, RETURN_TYPE
# It prints:
#   1) Your empirical breach stats (same as your script)
#   2) A GARCH(1,1) forecasted breach probability for the NEXT window
#      (vol clustering-adjusted tail risk estimate)
# ============================================================

def _window_to_trading_days(window: str) -> int:
    w = window.upper().strip()
    if w.endswith("W"):
        return 5 * int(w[:-1])
    if w.endswith("M"):
        # trading-day approximation; good enough for risk gating
        return 21 * int(w[:-1])
    raise ValueError(f"Unsupported WINDOW='{window}'. Use like '1W','2W','4W','1M','3M'.")

def _norm_cdf(z: float) -> float:
    # stable normal cdf without scipy
    return 0.5 * (1.0 + np.math.erf(z / np.sqrt(2.0)))

def _t_cdf(z: float, nu: float) -> float:
    # if scipy exists use it, else fallback to normal approx
    try:
        from scipy.stats import t
        return float(t.cdf(z, df=nu))
    except Exception:
        return _norm_cdf(z)

def _download_close_one(ticker: str, start: str, end: str | None, use_years: bool, years: int) -> pd.Series:
    if use_years:
        hist = yf.Ticker(ticker).history(period=f"{years}y", auto_adjust=True)
    else:
        hist = yf.Ticker(ticker).history(start=start, end=end, auto_adjust=True)

    if hist.empty or "Close" not in hist.columns:
        raise ValueError(f"No usable Close data returned for {ticker}.")

    close = hist["Close"].dropna().copy()
    close = close[close.index.dayofweek < 5]  # defensive weekdays only
    return close

def _empirical_window_returns(close: pd.Series, window: str, return_type: str) -> pd.Series:
    window_close = close.resample(window).last().dropna()
    if return_type.lower() == "log":
        r = np.log(window_close / window_close.shift(1))
    else:
        r = window_close.pct_change()
    return r.dropna().rename("ret")

def _fit_garch_forecast_prob(close: pd.Series, horizon_days: int, threshold: float, return_type: str) -> dict:
    """
    Fits GARCH(1,1) with Student-t errors to DAILY log returns.
    Then forecasts NEXT horizon variance and estimates P(R_h <= threshold).

    Notes:
    - We model log-returns internally for stability.
    - If RETURN_TYPE is "simple", we convert threshold to a log-return barrier:
        log(1+threshold)
      (valid for threshold > -1.0).
    """
    try:
        from arch import arch_model
    except Exception as e:
        return {
            "available": False,
            "reason": "Missing 'arch' package. Install: pip install arch",
            "error": str(e),
        }

    # daily log returns
    lr = np.log(close).diff().dropna()
    # arch uses percent scaling for numerical stability
    r_pct = (lr * 100.0).astype(float)

    am = arch_model(r_pct, mean="Constant", vol="GARCH", p=1, q=1, dist="t")
    res = am.fit(disp="off")

    mu_day_pct = float(res.params.get("mu", 0.0))
    nu = float(res.params.get("nu", np.nan))

    # 1..h allow 1-step-ahead recursion
    f = res.forecast(horizon=horizon_days, reindex=False)
    var_path_pct2 = f.variance.iloc[-1].values  # percent^2 per day
    var_h = float(var_path_pct2.sum() / (100.0**2))  # convert to log-return variance
    mu_h = float((mu_day_pct / 100.0) * horizon_days)  # log-return mean

    if var_h <= 0:
        return {"available": True, "prob_breach_garch": np.nan, "nu": nu, "mu_h": mu_h, "sigma_h": np.nan}

    # threshold to log-return barrier
    if return_type.lower() == "log":
        thr_lr = float(threshold)
    else:
        if threshold <= -1.0:
            raise ValueError("For simple returns, threshold must be > -100%.")
        thr_lr = float(np.log1p(threshold))

    sigma_h = float(np.sqrt(var_h))
    z = (thr_lr - mu_h) / sigma_h

    # Student-t if nu is valid, else normal fallback
    if np.isfinite(nu) and nu > 2:
        p = _t_cdf(z, nu)
        dist_used = "t"
    else:
        p = _norm_cdf(z)
        dist_used = "normal"

    return {
        "available": True,
        "prob_breach_garch": float(p),
        "dist_used": dist_used,
        "nu": None if (not np.isfinite(nu)) else float(nu),
        "mu_h": float(mu_h),
        "sigma_h": float(sigma_h),
        "sigma_h_pct": float(sigma_h * 100.0),
        "threshold_as_logret": float(thr_lr),
    }

# -----------------------------
# Run using YOUR existing params
# -----------------------------
close = _download_close_one(
    ticker=ticker,
    start=start_date,
    end=end_date,
    use_years=USE_YEARS,
    years=years,
)

# Empirical (your original logic)
window_ret = _empirical_window_returns(close, WINDOW, RETURN_TYPE)
total_periods = int(window_ret.shape[0])
count_breach = int((window_ret <= threshold).sum())
prob_breach = (count_breach / total_periods) if total_periods > 0 else np.nan

print(f"Ticker: {ticker}")
print(f"Window: {'last ' + str(years) + ' years' if USE_YEARS else f'{start_date} to {end_date or 'today'}'}")
print(f"\nReturn horizon: {WINDOW}  |  Return type: {RETURN_TYPE}")
print(f"Total periods analyzed: {total_periods}")
print(f"Periods with return ≤ {threshold:.1%}: {count_breach} ({prob_breach:.2%})")

print("\nSummary of windowed returns:")
print(window_ret.describe(percentiles=[0.01, 0.05, 0.10]))

print(f"\nMost recent periods breaching {threshold:.1%}:")
print(
    window_ret[window_ret <= threshold]
    .tail(10)
    .to_frame()
    .assign(ret_pct=lambda x: (x["ret"] * 100).round(2))
)

# GARCH forecast (NEXT window)
h_days = _window_to_trading_days(WINDOW)
garch_out = _fit_garch_forecast_prob(close, horizon_days=h_days, threshold=threshold, return_type=RETURN_TYPE)

print("\n" + "-" * 60)
print("GARCH(1,1) forecast for NEXT window (vol clustering-adjusted)")
if not garch_out["available"]:
    print(f"ARCH/GARCH not available: {garch_out.get('reason')}")
else:
    print(f"Horizon trading days: {h_days}")
    print(f"Dist used: {garch_out['dist_used']}  |  nu: {garch_out.get('nu')}")
    print(f"Forecast sigma_h (log-ret std): {garch_out['sigma_h_pct']:.2f}%")
    print(f"P(next-window return ≤ {threshold:.1%}) ≈ {garch_out['prob_breach_garch']:.2%}")

# Optional: quick regime flag (useful for gating premium selling)
if garch_out.get("available") and np.isfinite(garch_out.get("prob_breach_garch", np.nan)):
    # Example rule: if forecast tail prob is meaningfully above empirical, you're in a worse regime
    uplift = garch_out["prob_breach_garch"] - prob_breach if np.isfinite(prob_breach) else np.nan
    if np.isfinite(uplift):
        print(f"Tail-risk uplift vs empirical: {uplift:+.2%}")

Ticker: AMZ
Window: 2010-01-01 to today

Return horizon: 1W  |  Return type: simple
Total periods analyzed: 417
Periods with return ≤ -7.8%: 9 (2.16%)

Summary of windowed returns:
count    417.000000
mean       0.004505
std        0.063736
min       -0.998843
1%        -0.106060
5%        -0.058390
10%       -0.043005
50%        0.004848
max        0.169993
Name: ret, dtype: float64

Most recent periods breaching -7.8%:
                                ret  ret_pct
Date                                        
2010-07-04 00:00:00-04:00 -0.113653   -11.37
2011-08-07 00:00:00-04:00 -0.093820    -9.38
2011-08-21 00:00:00-04:00 -0.086773    -8.68
2011-10-30 00:00:00-04:00 -0.109932   -10.99
2014-07-27 00:00:00-04:00 -0.098364    -9.84
2016-01-10 00:00:00-05:00 -0.107526   -10.75
2016-02-07 00:00:00-05:00 -0.153710   -15.37
2017-08-06 00:00:00-04:00 -0.090845    -9.08
2018-05-27 00:00:00-04:00 -0.998843   -99.88

------------------------------------------------------------
GARCH(1,1) forecas

In [9]:

# -----------------------------
# USER INPUTS
# -----------------------------
TICKERS = ["SPY", "SGOL", "ELV", "USAR", "^VIX", "AMZN","TLT"]
START = "2010-01-01"
END = "2025-12-31"

# Choose: "D" = daily, "W" = weekly, "M" = monthly
FREQ = "D"

RETURN_TYPE = "simple"   # "log" or "simple"

# -----------------------------
# DOWNLOAD PRICES
# -----------------------------
prices = (
    yf.download(TICKERS, start=START, end=END, auto_adjust=True)["Close"]
    .dropna()
)

# -----------------------------
# RESAMPLE PRICES
# -----------------------------
prices = prices.resample(FREQ).last()

# -----------------------------
# COMPUTE RETURNS
# -----------------------------
if RETURN_TYPE == "log":
    returns = np.log(prices / prices.shift(1))
else:
    returns = prices.pct_change()

returns = returns.dropna()

# -----------------------------
# CORRELATION MATRIX
# -----------------------------
corr_matrix = returns.corr()

display(corr_matrix)



[*********************100%***********************]  7 of 7 completed
C:\Users\Jonbf5\AppData\Local\Temp\ipykernel_37576\4244002712.py:32: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change()


Ticker,AMZN,ELV,SGOL,SPY,TLT,USAR,^VIX
Ticker,,,,,,,
AMZN,1.000000,0.008405,-0.018918,0.702997,0.011011,-0.015070,-0.563044
ELV,0.008405,1.000000,-0.003387,0.136844,0.054102,0.040147,-0.120031
SGOL,-0.018918,-0.003387,1.000000,0.109665,0.154584,0.062637,-0.057341
SPY,0.702997,0.136844,0.109665,1.000000,0.125560,0.009033,-0.788811
TLT,0.011011,0.054102,0.154584,0.125560,1.000000,-0.017281,-0.042891
USAR,-0.015070,0.040147,0.062637,0.009033,-0.017281,1.000000,-0.049697
^VIX,-0.563044,-0.120031,-0.057341,-0.788811,-0.042891,-0.049697,1.000000
